In [1]:
import pandas as pd
import pixiedust
import numpy as np
import random
import requests
import json

Pixiedust database opened successfully


In [2]:
df = pd.read_csv('DB_ready.csv')

In [3]:
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")
params = {'include_docs': 'true'}

In [4]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_all_docs".format(**config)

In [5]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.get(url=url, params=params)
data = json.loads(r.text)

In [6]:
print(json.dumps(data, indent=2))

{
  "total_rows": 1135,
  "offset": 0,
  "rows": [
    {
      "id": "88b2a4331fa1b0d9be1594ac6c000bd3",
      "key": "88b2a4331fa1b0d9be1594ac6c000bd3",
      "value": {
        "rev": "2-ecca81292e8562dda6b3bcd641dbe7e2"
      },
      "doc": {
        "_id": "88b2a4331fa1b0d9be1594ac6c000bd3",
        "_rev": "2-ecca81292e8562dda6b3bcd641dbe7e2",
        "": "907",
        "Items": "Plastic rolls (UNCHR)",
        "Origin": "UNHCR",
        "Items grouped": "Plastic rolls (UNCHR)",
        "Item group filter": "",
        "Category": "Shelter",
        "Unit": "unit",
        "amount": "1.0",
        "Mission": "MISSION 17/5 *01",
        "District": "Gorkha",
        "VDC": "Chhoprak",
        "Village/Ward/extra info": "Gaira, Jaisithok, Phirphire, Arubot, ward 8",
        "lat": "28.0556",
        "lon": "84.5881",
        "District Latitude": "28.33333",
        "District Longitude": "84.83333",
        "Date": "2015-05-17",
        "Location name": "Chhoprak, Gaira, Jaisithok, 

In [7]:
df = [row["doc"] for row in data["rows"]]

In [8]:
df =pd.DataFrame(df)

In [9]:
df[["lat","lon","amount"]] = df[["lat","lon","amount"]].apply(pd.to_numeric)

In [10]:
#df[["amount"]] = df[["amount"]].astype(int)

In [11]:
display(df)

In [12]:
people = pd.DataFrame(columns=['name','needed_item','amount','location','unit'])


In [13]:
with open('people.txt','r') as f:
    names = f.readlines()

In [14]:
names = [n.strip('\n') for n in names]
people['name']= names

In [15]:
items_list = set([(doc["Items"].lower(), doc["Unit"]) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc])
locations = [(doc["Location name"], doc["lat"], doc["lon"]) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc]

In [16]:
location_dict = {' '.join(location[:-2]): (location[-2],location[-1]) for location in locations}

In [17]:
unit_lookup = {name:unit for name, unit in items_list}

In [18]:
people['needed_item'] = [random.choice(list(items_list))[0] for i in range(len(people))]

In [19]:
people['unit']=[unit_lookup[item] for item in people['needed_item']]

In [20]:
items_list

{('acenac (muscular pains)', 'tabs'),
 ('adhesive tape', 'rolls'),
 ('azifast (amoxil)', 'tabs'),
 ('bandage', 'unit'),
 ('beaten rice', 'sack'),
 ('betadine', 'bottle'),
 ('biscuits', 'box'),
 ('blankets', 'unit'),
 ('brucet', 'tabs'),
 ('brufen', 'bottle'),
 ('brufen', 'tabs'),
 ('children milk', 'pack'),
 ('colour pencils', 'pack'),
 ('colour pencils', 'unit'),
 ('cotton', 'pack'),
 ('crepe bandage', 'rolls'),
 ('dal', 'pack'),
 ('dal', 'sack'),
 ('dalmot', 'pack'),
 ('dalmot', 'sack'),
 ('decold', 'tabs'),
 ('dettol antiseptic liquids', 'bottle'),
 ('dettol soaps', 'unit'),
 ('diapers', 'box'),
 ('diapers', 'unit'),
 ('diclofenac gel', 'unit'),
 ('electrobin', 'unit'),
 ('electrolytes', 'unit'),
 ('eraser', 'unit'),
 ('feeding bottles', 'unit'),
 ('flex on (painkiller', 'box'),
 ('flexon', 'unit'),
 ('foam pads', 'unit'),
 ('food', 'pack'),
 ('food packages', 'unit'),
 ('formula milk', 'box'),
 ('fruit based energy drinks', 'box'),
 ('furandanal', 'pack'),
 ('gauge bandage', 'rolls

In [21]:
locations = pd.DataFrame([(doc["Location name"], (doc["lat"],doc["lon"])) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc], columns = ['name','latlon'])

In [22]:
people['amount'] = np.random.randint(5, size = len(people))+1

In [23]:
people['location'] = [random.choice(list(locations['name'])) for i in range(len(people))]

In [24]:
station_supplies = pd.DataFrame(columns=['name','item_name','item_quanity','lat','lon'])

In [25]:
station_supplies = {}

for person in people.iterrows():
    person = person[1]
    if (person['needed_item'],person['location']) in station_supplies.keys():
        station_supplies[(person['needed_item'],person['location'])] += person['amount']
    else:
        station_supplies[(person['needed_item'],person['location'])] = person['amount']

In [26]:

station_supplies

{('noodles purchased (75 bags of 20 units)', 'Chhetrapa, Yabo'): 3,
 ('masks', 'Dolakha, '): 4,
 ('matress', 'Marpak, '): 4,
 ('electrolytes',
  'Dhuyang, Ghumtang, Dandagaun, Kolbari, Kimbegaun, Khanyabas, Devchagaun, Chapleti'): 5,
 ('jeevanjal', 'Shreenathkot, Hansapur'): 5,
 ('paraflam', 'Hagam, Yanlakot'): 2,
 ('plastic rolls (unchr)', 'Chapagaun, Neupanegaun'): 2,
 ('mosquito nets', 'Gotikhel, '): 1,
 ('sanitary pads', 'Lisankhu, Wafal'): 1,
 ('tent (12x15)', 'Tandrang, Darme ward 7'): 3,
 ('paraflam', 'Bhardev, Bhardeudeldanda'): 3,
 ('medicine - surgical', 'Kakani, Ranipauwa'): 2,
 ('medicine - supplies', 'Ramkot, Nagarjun'): 4,
 ('crepe bandage', 'Sikre, '): 3,
 ('pasta-thick', 'Reegaun, Satya'): 1,
 ('colour pencils', 'Ramche, Battay, ChamKharkay, Bhirkuna, Binday'): 2,
 ('sanitary pads', 'Tupche, Mahadev Khadka, ward 7'): 3,
 ('nylon rope', 'Dhuyang, Deurali Chalasi ward 6'): 5,
 ('medicine - supplies', 'Chapagaun, Jhalungtar'): 5,
 ('gauze dressings', 'Jharuwarasi, and also

In [27]:
matrix = []

for key in station_supplies.keys():
    item = key[0]
    location = key[1]
    amount = station_supplies[key]
    matrix.append(pd.Series([item, location, amount, unit_lookup[item], *location_dict[location]],index=['needed_item','Location','amount needed', 'unit', 'lat', 'long']))

In [28]:
final_data = pd.DataFrame(matrix)

In [48]:
final_data.head()

,needed_item,Location,amount needed,unit,lat,long
0,noodles purchased (75 bags of 20 units),"Chhetrapa, Yabo",3,box,27.66913,86.13334
1,masks,"Dolakha,",4,box,27.83333,86.25
2,matress,"Marpak,",4,unit,28.0135,84.9646
3,electrolytes,"Dhuyang, Ghumtang, Dandagaun, Kolbari, Kimbega...",5,unit,27.85199,85.85637
4,jeevanjal,"Shreenathkot, Hansapur",5,unit,28.087,84.6393


In [29]:
display(final_data)

In [66]:
new_df = []
for location_df in list(df.groupby('Location name')):
    temp_df=location_df[1][['amount','Items']]
    temp_df['Location name'] = location_df[0]
    temp_df.rename(columns={'amount':'current amount'}, inplace=True)
    new_df.append(temp_df)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/andrew/.local/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [69]:
current_amounts = pd.concat(new_df)
current_amounts

,current amount,Items,Location name
799,2.0,Biscuits,"Aalapot, Kageshwari, Halapur"
800,2.0,Noodles,"Aalapot, Kageshwari, Halapur"
801,1.0,Rice,"Aalapot, Kageshwari, Halapur"
802,1.0,Medical Kit,"Aalapot, Kageshwari, Halapur"
803,80.0,Medicine - rehydration,"Aalapot, Kageshwari, Halapur"
804,5.0,Piyush,"Aalapot, Kageshwari, Halapur"
805,13.0,Plastic rolls,"Aalapot, Kageshwari, Halapur"
846,1.0,Medical Kit,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
847,60.0,Medicine - rehydration,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
848,1.0,Sanitary pads,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"


,Items,Location name
799,Biscuits,"Aalapot, Kageshwari, Halapur"
800,Noodles,"Aalapot, Kageshwari, Halapur"
801,Rice,"Aalapot, Kageshwari, Halapur"
802,Medical Kit,"Aalapot, Kageshwari, Halapur"
803,Medicine - rehydration,"Aalapot, Kageshwari, Halapur"
804,Piyush,"Aalapot, Kageshwari, Halapur"
805,Plastic rolls,"Aalapot, Kageshwari, Halapur"
846,Medical Kit,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
847,Medicine - rehydration,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
848,Sanitary pads,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
